## Ejercicio 3. 

Nos piden aplicar las funciones para:

***[1]***: Calcular el autovector asociado al autovalor más chico de la matríz L.

***[2]***: Calcular el autovector asociado al autovalor más grande de R.

Además, interpretar los valores que nos dan en términos del grafo.

También nos piden calcular los vectores ***s*** asociados a estos autovectores y comparar con la particion esperada con el grafo de ejemplo.


In [1]:
import numpy as np
from linalg import *

In [2]:
# Grafo de ejemplo
A_sample = np.array([
    [0, 1, 1, 1, 0, 0, 0, 0],
    [1, 0, 1, 1, 0, 0, 0, 0],
    [1, 1, 0, 1, 0, 0, 0, 0],
    [1, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 1, 1],
    [0, 0, 0, 0, 1, 0, 1, 1],
    [0, 0, 0, 0, 1, 1, 0, 1],
    [0, 0, 0, 0, 1, 1, 1, 0]
])

# Función para obtener el vector s
comunidades = lambda autovector: [1 if x > 0 else -1 for x in autovector] 

***[1]*** Calculamos el autovector asociado al segundo autovalor más chico de L ($\lambda>0$)

Este $\lambda_{min}$ esta asociado a un autovector $v_{min} \in \Re^{N}$ que se interpreta como.
$$\begin{cases}
    sign(v_i) = 1 & \text{ si museo}_i \in \text{comunidadA} \\
    sign(v_i) = -1 & \text{ si museo}_i \in \text{comunidadB} \\
\end{cases}$$ 

***N***: cantidad de musos

<center> <img src='img/comunidades_L.png' width=500, height=300> </center>

In [3]:
L_sample = calcula_L(A_sample)
segundo_autovalor_minimo_L, segundo_autovector_minimo_L = metpotI2(L_sample, mu=1e-5)

print("Segundo autovalor mínimo de L:", segundo_autovalor_minimo_L)
print("Comunidades (s): ", comunidades(segundo_autovector_minimo_L))

Segundo autovalor mínimo de L: 9.999999999919936e-06
Comunidades (s):  [-1, -1, -1, -1, 1, 1, 1, 1]


***[2]***: Calcular el autovector asociado al autovalor más grande de R.

Este $\lambda_{max}$ esta asociado a un autovector $v_ \in \Re^{N}$ que se interpreta como.
$$\begin{cases}
    sign(v_i) = 1 & \text{ si museo}_i \in \text{comunidadA} \\
    sign(v_i) = -1 & \text{ si museo}_i \in \text{comunidadB} \\
\end{cases}$$ 

<center> <img src='img/comunidades_R.png' width=500, height=300> </center>

In [4]:
R_sample = calcula_R(A_sample)
autovalor_maximo_R, autovector_maximo_R = metpot(R_sample)

print("Autovalor máximo para R:", autovalor_maximo_R)
print("Comunidades (s): ", comunidades(autovector_maximo_R))

Autovalor máximo para R: -2.838660118048165
Comunidades (s):  [1, 1, 1, 1, 1, 1, 1, 1]
